In [3]:
import os
import openai
import pandas as pd
import numpy as np

openai.api_key = os.getenv("OPENAI_API_KEY")

# Process the csv into jsonl

In [48]:
df = pd.read_csv("mturk_processed - combined.csv")
df = df[df.valid_annot1==1]
df.head()

,startphrase,ending1,ending2,labels,valid_annot1,valid_annot2,valid_annot3,valid_2+,valid_all3
0,the ocean waves forced their way unto the beac...,The ocean waves are soothing and healing,The ocean waves are dangerous and can kill you,0,1,1,1,1,1
1,the ocean waves forced their way unto the beac...,The ocean waves are soothing and healing,The ocean waves are dangerous and can kill you,1,1,1,1,1,1
2,the teach said that the student had the brain ...,The student is dumb,The student is smart,0,1,1,1,1,1
3,the teach said that the student had the brain ...,The student is dumb,The student is smart,1,1,1,1,1,1
6,i have the musical ability of a chipmunk,My music sounds scratchy and random,My music is soothing and melodious,0,1,1,1,1,1


In [53]:
from sample_metaphors import trial_dataset
df = pd.DataFrame(trial_dataset['test'])
df.head()

,startphrase,ending1,ending2,label
0,He should be back before more sunlight reaches us,He will be back quickly,He will be back slowly,0
1,He should be back before the sun implodes,He will be back quickly,He will be back slowly,1
2,Her commitment to the cause was as sturdy as oak,She was committed,She was uncommitted,0
3,Her commitment to the cause was as sturdy as p...,She was committed,She was uncommitted,1
4,The crowd's reaction to the performance fizzle...,The crowd was enthusiastic,The crowd was unenthusiastic,0


In [66]:
debugging = False
model_name = 'curie'
assert model_name in ['ada', 'babbage', 'curie', 'davinci']
json_lines = []
scores = []


if not debugging:
    response = input(f"about the spend $$$ on openai API (model {model_name})! conitnue? [y/n]")
    if response.lower() != 'y':
        raise Exception("Not continuing.")
else:
    print('just debugging. this is free.')

    
for i, line in df.iterrows():
    if not i%10:
        print(i)
    query = line['startphrase']
    doc1 = line['ending1']
    doc2 = line['ending2']
    
    if debugging:
        res = {
          "data": [
            {
              "document": 0,
              "object": "search_result",
              "score": 4.643+i
            },
            {
              "document": 1,
              "object": "search_result",
              "score": 5.402+i
            }
          ],
          "model": "ada:2020-05-03",
          "object": "list"
        }
    else:
        res = openai.Engine(model_name).search(
            search_model=model_name, 
            query=query, 
            max_rerank=5,
            documents=[doc1, doc2]
        )

    if res['data'][1]['document'] > res['data'][0]['document']:
        s = (res['data'][0]['score'], res['data'][1]['score'])
    else:
        print('this is weird but it actually happened')
        s = (res['data'][1]['score'], res['data'][0]['score'])

    scores.append(s)
    
df.loc[:, 'score_ending1'] = list(zip(*scores))[0]
df.loc[:, 'score_ending2'] = list(zip(*scores))[1]

df.to_csv(f"trial_dataset_predictions_{model_name}.csv", index=False)

about the spend $$$ on openai API (model curie)! conitnue? [y/n]y
0
10
20
30


In [67]:
df.head()

,startphrase,ending1,ending2,label,score_ending1,score_ending2
0,He should be back before more sunlight reaches us,He will be back quickly,He will be back slowly,0,98.176,84.007
1,He should be back before the sun implodes,He will be back quickly,He will be back slowly,1,114.603,110.924
2,Her commitment to the cause was as sturdy as oak,She was committed,She was uncommitted,0,121.905,95.759
3,Her commitment to the cause was as sturdy as p...,She was committed,She was uncommitted,1,101.636,89.566
4,The crowd's reaction to the performance fizzle...,The crowd was enthusiastic,The crowd was unenthusiastic,0,87.064,91.341


In [68]:
pred = df['score_ending2'] > df['score_ending1']
try:
    correct = (pred == df['labels']).tolist()
except KeyError:
    correct = (pred == df['label']).tolist()

print(model_name)
print("accuracy is:", sum(correct)/len(correct))

curie
accuracy is: 0.6052631578947368
